In [ ]:
import pandas as pd

# Data Loading

We have to copy & paste the `load_adult_data` function from the 1st notebook.

In [ ]:
def load_adult_data(data_file='../data/adult_data.csv'):
    COLUMN_NAMES = (
        'age',
        'workclass',
        'fnlwgt',
        'education',
        'education_num',
        'marital_status',
        'occupation',
        'relationship',
        'race',
        'sex',
        'capital_gain',
        'capital_loss',
        'hours_per_week',
        'native_country',
        'income',
    )
    
    return pd.read_csv(
        data_file,
        names=COLUMN_NAMES,
        skipinitialspace=True
    )

In [ ]:
adult_df = load_adult_data()

# Preprocessing

We have to copy & paste the functions from the 2nd notebook.

In [ ]:
def add_age_group(adult_df):
    age_group = pd.cut(
        adult_df['age'],
        bins=range(10, 101, 10),
        right=False,
        labels=[f'{age_start}~{age_start + 9}'
                for age_start in range(10, 100, 10)]
    )
    
    return adult_df.assign(age_group=age_group)

In [ ]:
def change_education_type_to_category(adult_df):
    education_order = (
        adult_df
        .groupby('education')['education_num']
        .unique()
        .sort_values()
        .index
    )
    
    return adult_df.astype({
        "education": pd.CategoricalDtype(categories=education_order,
                                         ordered=True),
        "education_num": pd.CategoricalDtype(ordered=True),
    })

In [ ]:
adult_df = (  # We can easily add other preprocessing in this way
    adult_df
    .pipe(add_age_group)
    .pipe(change_education_type_to_category)
    # .pipe(yet_another_preprocessing)
)

# Equivalent to
# adult_df = add_age_group(adult_df)
# adult_df = change_education_type_to_category(adult_df)

# EDA (Exploratory Data Analysis)

![EDA](../image/eda.png)

Refer to [this GitHub repo](https://github.com/sesise0307/pydata2021-eda) for a detailed tutorial.

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = 10, 5  # Set a default figure size
sns.set_style('whitegrid')  # Set a default figure style

In [ ]:
numeric_columns = adult_df.select_dtypes(include='number').columns
numeric_columns

In [ ]:
string_columns = adult_df.select_dtypes(exclude='number').columns
string_columns

## Histogram / KDE / Boxplot

In [ ]:
plt.subplot(131)
sns.histplot(data=adult_df, x='age', hue='income', multiple="stack")

plt.subplot(132)
sns.kdeplot(data=adult_df, x='age', hue='income', multiple="stack")

plt.subplot(133)
sns.boxplot(data=adult_df, y='age', x='income');

What about against 'hours_per_week'?

In [ ]:
plt.subplot(131)
sns.histplot(data=adult_df, x='hours_per_week', hue='income', multiple="stack")

plt.subplot(132)
sns.kdeplot(data=adult_df, x='hours_per_week', hue='income', multiple="stack")

plt.subplot(133)
sns.boxplot(data=adult_df, y='hours_per_week', x='income');

What about other variables?

Let's define a function.

In [ ]:
def statistical_plots(data, var, separate_by="income"):
    plt.subplot(131)
    sns.histplot(data=data, x=var, hue=separate_by, multiple="stack")

    plt.subplot(132)
    sns.kdeplot(data=data, x=var, hue=separate_by, multiple="stack")

    plt.subplot(133)
    sns.boxplot(data=data, y=var, x=separate_by)    

In [ ]:
statistical_plots(adult_df, "fnlwgt")

In [ ]:
statistical_plots(adult_df, "hours_per_week", separate_by="sex")

In [ ]:
@widgets.interact(
    var=numeric_columns,
    separate_by=string_columns,    
)
def interactive_statistical_plots(var, separate_by="income"):
    return statistical_plots(adult_df, var, separate_by)

## Count Heatmap

In [ ]:
def normalized_count_heatmap(adult_df, var, separate_by="income"):
    sns.heatmap(
        adult_df.groupby(separate_by)[[var]].value_counts(normalize=True).unstack(),
        cmap='Blues',
        annot=True,
        linewidths=1,
        vmin=0.0,
        vmax=1.0,
        fmt='.1%'
    )

In [ ]:
@widgets.interact(
    var=string_columns,
    separate_by=string_columns,    
)
def wrapper(var="race", separate_by="income"):
    return normalized_count_heatmap(adult_df, var, separate_by)